### Simple Decorator

In [1]:
def add_stars(func): # This function add_stars will receive a function as an input
    def func_that_gets_returned(*args, **kwargs): 
        # This name doesn't matter. It's what the decorated function gets transformed into
        # Note that *args and **kwargs could be anything. Doing this allows the decorated function to 
        # have any signature, and this decorator will allow it to work
        print('*'*15)
        func(*args, **kwargs) # execute the function like normal
        print('*'*15)
    return func_that_gets_returned

def print_something(phrase):
    print(phrase)
    

print_something("undecorated")

undecorated


In [2]:
def add_stars(func): # This function add_stars will receive a function as an input
    def func_that_gets_returned(*args, **kwargs): 
        # This name doesn't matter. It's what the decorated function gets transformed into
        # Note that *args and **kwargs could be anything. Doing this allows the decorated function to 
        # have any signature, and this decorator will allow it to work
        print('*'*15)
        func(*args, **kwargs) # execute the function like normal
        print('*'*15)
    return func_that_gets_returned

@add_stars
def print_something(phrase):
    print(phrase)
    

print_something("decorated")

***************
decorated
***************


### Error when inner func doesn't account for function signature of the original function

In [5]:
def add_stars(func): # This function add_stars will receive a function as an input
    def func_that_gets_returned(): 
        # This name doesn't matter. It's what the decorated function gets transformed into
        # have any signature, and this decorator will allow it to work
        print('*'*15)
        func() # execute the function like normal
        print('*'*15)
    return func_that_gets_returned

@add_stars
def print_something(phrase):
    print(phrase)
    

print_something("decorated")

TypeError: add_stars.<locals>.func_that_gets_returned() takes 0 positional arguments but 1 was given

### What you have to do without decorators

In [8]:
def add_stars(func): # This function add_stars will receive a function as an input
    def func_that_gets_returned(*args, **kwargs): 
        # This name doesn't matter. It's what the decorated function gets transformed into
        # Note that *args and **kwargs could be anything. Doing this allows the decorated function to 
        # have any signature, and this decorator will allow it to work
        print('*'*15)
        func(*args, **kwargs) # execute the function like normal
        print('*'*15)
    return func_that_gets_returned

def print_something(phrase):
    print(phrase)
    

# You could make a new function if you want to avoid clobbering the name of `print_something`
print_something_with_stars = add_stars(print_something)
print_something_with_stars("with stars")    

print_something = add_stars(print_something)
# This is un-pythonic, but shows you exactly what happens
# The `print_something` function no longer exists as it used to.


print_something("decorated without the @ sign!")

***************
with stars
***************
***************
decorated without the @ sign!
***************


# DECORATORS WITH CLASSES

### The below add_stars decorator is unchanged, and still works. The signature is misleading, because `func` is really a class object commonly abbreviated as `cls`. Either way, the args/kwargs will be passed like before. 

In [12]:
def add_stars(func): # This function add_stars will receive a function as an input
    def func_that_gets_returned(*args, **kwargs): 
        # This name doesn't matter. It's what the decorated function gets transformed into
        # Note that *args and **kwargs could be anything. Doing this allows the decorated function to 
        # have any signature, and this decorator will allow it to work
        print('*'*15)
        func(*args, **kwargs) # execute the function like normal
        print('*'*15)
    return func_that_gets_returned

class A:
    def __init__(self,a):
        self.a=a
        print("in class constructor")

a = A(1)


print('\n'*3)

@add_stars
class B:
    def __init__(self,a):
        self.a=a
        print("in class constructor with stars around it")

b = B(1)

in class constructor




***************
in class constructor with stars around it
***************


### Important^ See how args and kwargs (in the above case, just the parameter `a`) are passed to the decorated class still, becuase the new class definition is `func_that_gets_returned`. First `func_that_gets_returned` prints stars, then it passes all your args/kwargs to whatever definition it just overwrote, then whenever the code kicked off by the constructor is done, more stars are printed. Cool!

## Now lets make a decorator that takes in parameters. In this case, such a decorator is really just a function that returns a decorator, and that decorator takes in the function/class below it as an input...

In [31]:
def add_stars(num_stars):
    def the_actual_class_decorator(cls):
        def func_that_gets_returned(*args, **kwargs): 
            print('*'*num_stars)
            cls(*args, **kwargs) # execute the class instantiation like normal
            print('*'*num_stars)
        return func_that_gets_returned # the decorator returns this 
    return the_actual_class_decorator

class A:
    def __init__(self,a):
        self.a=a
        print("in class constructor")

a = A(1)


print('\n'*3)

@add_stars(4)
class B:
    def __init__(self,a):
        self.a=a
        print("in class constructor with stars around it")

b = B(1)

in class constructor




****
in class constructor with stars around it
****


# INSANITY BELOW, WHICH IS WRONG AND I DON"T UNDERSTAND IT YET

### Okay so apparently decorators can be any old object, not just functions. If an object is used as a decorator, it must define a __call__ method. When an object instance is called like a function, the __call__ method is automatically invoked.

In [2]:
class DecoratorClass:
    def __init__(self, func): 
        # the DecoratorClass instance receives `say_hello` as its `func`. 
        # Note that args and kwargs don't even exist yet. This is just defining the skeleton.
        self.func = func

    def __call__(self, *args, **kwargs): # when say_hello receives args,kwargs, they appear in the __call__ method
        print("Before calling the function.")
        result = self.func(*args, **kwargs)
        print("After calling the function.")
        return result

@DecoratorClass
def say_hello(name):
    print(f"Hello, {name}!")

say_hello("Alice")

Before calling the function.
Hello, Alice!
After calling the function.


### Let's make a decorator that decorates a function. The function it decorates represents the name of some sort of attribute. The desired behavior is that when someone accesses this weird-function-attribute hybrid as if it were an instance attribute, its value should be returned.

### In order to do that, we need to implement `__get__`.

### The `__get__` method is what makes Property a descriptor. Descriptors are objects that define the behavior of attributes when they're accessed as object attributes (like `obj.attr`). If a class has a `__get__` method, Python will use that method to get the value of the attribute when it's accessed, instead of just returning the value of the attribute directly.

In [ ]:
### THE BW

In [26]:
#self=SomeProperty instance, instance=Test instance, owner=Test class
class SomeProperty:
    def __init__(self, getter, setter=None):
        self.getter = getter
        self.setter = setter

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.getter(instance)

    def __set__(self, instance, value):
        if not self.setter:
            raise AttributeError("can't set attribute")
        self.setter(instance, value)

    def setter(self, setter):
        return type(self)(self.getter, setter)


class Test:
    def __init__(self):
        self._x = 0

    @SomeProperty
    def x(self):
        return self._x

    @x.setter
    def x(self, value):
        if value < 0:
            raise ValueError("Must be non-negative")
        self._x = value
    
t = Test()
t.x
    
t = Test()
print(f"{vars(t)=}")
print(t.x)
t.x = 5
print(f"{vars(t)=}")
print(t.__dict__)


TypeError: 'NoneType' object is not callable

In [18]:
type(t.x)

int

In [9]:
class Property:
    def __init__(self, fget):
        self.fget = fget

    def __get__(self, instance, owner):
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(instance)


class Test:
    def __init__(self):
        self._x = 0

    @Property
    def x(self):
        return self._x
    
t = Test()
t.x

0